<a href="https://colab.research.google.com/github/Alvald1/HSE_colab/blob/main/17_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Использование СУБД SQLite

SQLite - компактная встраиваемая СУБД. SQLite хранит всю базу данных (включая определения, таблицы, индексы и данные) в единственном стандартном файле на том компьютере, на котором исполняется программа. https://ru.wikipedia.org/wiki/SQLite

In [ ]:
import sqlite3
 
conn = sqlite3.connect("mydatabase.db") # или :memory: чтобы сохранить в RAM
cursor = conn.cursor()

SQL — декларативный язык программирования, применяемый для создания, модификации и управления данными в реляционной базе данных. https://ru.wikipedia.org/wiki/SQL

Основной набор операций:
* создание в базе данных новой таблицы;
* добавление в таблицу новых записей;
* изменение записей;
* удаление записей;
* выборка записей из одной или нескольких таблиц (в соответствии с заданным условием);
* изменение структур таблиц.

### Создание новой таблицы

In [ ]:
import sqlite3
 
conn = sqlite3.connect("mydatabase.db") # или :memory: чтобы сохранить в RAM
cursor = conn.cursor()
# Создание таблицы
cursor.execute("""CREATE TABLE albums
                  (artist, title text, label text,
                   units_mln int, year int)
               """)

# cursor.execute("""DROP TABLE albums""")

### Удаление таблицы 

In [ ]:
cursor.execute("""DROP TABLE albums""")

In [ ]:
cursor.execute("""DROP TABLE IF EXISTS albums""")

In [ ]:
cursor.execute("""CREATE TABLE albums
                  (id INTEGER PRIMARY KEY, artist, title text, label text,
                   units_mln int, year int)
               """)

### Добавление данных

In [ ]:
# Вставляем данные в таблицу
cursor.execute(
    """INSERT INTO albums(artist, title, label, units_mln, year) VALUES (?, ?, ?, ?, ?)""", 
    ('Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979)
)# Безопасный метод "?"

# Сохраняем изменения
conn.commit()

In [ ]:
# Вставляем множество данных в таблицу используя безопасный метод "?"
albums = [('Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979),
          ('Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979),
          ('Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979)]
 
cursor.executemany(
    "INSERT INTO albums(artist, title, label, units_mln, year) VALUES (?,?,?,?,?)", 
    albums
)
conn.commit()

### Выборка данных

In [ ]:
for row in cursor.execute("SELECT * FROM albums"):
    print(row)

(1, 'Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979)
(2, 'Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979)
(3, 'Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979)
(4, 'Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979)


### Изменение записи

In [ ]:
cursor.execute("UPDATE albums SET year=? WHERE id=?",(1970, 3))

In [ ]:
# Получение списка значений
cursor.execute("SELECT * FROM albums").fetchall()

[(1, 'Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979),
 (2, 'Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979),
 (3, 'Pink Floyd', 'The Wall', 'Columbia Records', 23, 1970),
 (4, 'Pink Floyd', 'The Wall', 'Columbia Records', 23, 1979)]

In [ ]:
sql = """
UPDATE albums 
SET artist = 'John Doe' 
WHERE artist = 'Pink Floyd'
"""

cursor.execute(sql)
conn.commit()

In [ ]:
cursor.execute("SELECT * FROM albums").fetchall()

[(1, 'John Doe', 'The Wall', 'Columbia Records', 23, 1979),
 (2, 'John Doe', 'The Wall', 'Columbia Records', 23, 1979),
 (3, 'John Doe', 'The Wall', 'Columbia Records', 23, 1970)]

### Удаление записи

In [ ]:
sql = "DELETE FROM albums WHERE id = ?"
 
cursor.execute(sql, (4,))
conn.commit()

In [ ]:
cursor.execute("""DROP TABLE albums""")

## Экспорт данных из DataFrame

In [ ]:
import pandas as pd
data = pd.read_excel('17_art.xls', index_col='Unnamed: 0')

In [ ]:
data.head()

,artist,title,label,units_mln,year
0,Phil Collins «No Jacket Require,Phil Collins «No Jacket Require,Atlantic Records,12,1985
1,Matchbox Twenty «Yourself or Someone Like Yo,Matchbox Twenty «Yourself or Someone Like Yo,Atlantic Records,12,1996
2,Led Zeppelin «Led Zeppelin I,Led Zeppelin «Led Zeppelin I,Atlantic Records,12,1969
3,Kenny Rogers «Kenny Rogers’ Greatest Hit,Kenny Rogers «Kenny Rogers’ Greatest Hit,Atlantic Records,12,1980
4,Kenny G «Breathles,Kenny G «Breathles,Arista Records,12,1991


In [ ]:
data.to_sql('albums',conn)

In [ ]:
SQLres = cursor.execute("""
SELECT * FROM albums 
WHERE `year` > 1980 
AND units_mln <33
ORDER BY units_mln DESC 
LIMIT 0, 5""" )


In [ ]:
for row in SQLres:
    print(row)

(48, 'Eagles «Their Greatest Hits 1971-197', 'agles «Their Greatest Hits 1971-197', 'Rhino', 29, 1986)
(47, 'Billy Joel «Greatest Hits Volume I\xa0& Volume I', 'illy Joel «Greatest Hits Volume I\xa0& Volume I', 'Columbia Records', 23, 1985)
(43, 'Garth Brooks «Double Liv', 'arth Brooks «Double Liv', 'Capitol Nashville', 21, 1998)
(41, 'Shania Twain «Come on\xa0Ove', 'hania Twain «Come on\xa0Ove', 'Mercury Nashville', 20, 1987)
(42, 'Fleetwood Mac «Rumour', 'leetwood Mac «Rumour', 'Warner Bros.', 20, 1987)


In [ ]:
for row in cursor.execute("""
SELECT * FROM `albums` 
WHERE `year` > 1976  
ORDER BY `units_mln` DESC LIMIT 0, 10"""):
    print(row)

(49, 'Michael Jackson «Thrille', 'ichael Jackson «Thrille', 'Epic Records', 33, 1982)
(48, 'Eagles «Their Greatest Hits 1971-197', 'agles «Their Greatest Hits 1971-197', 'Rhino', 29, 1986)
(45, 'Pink Floyd «The Wal', 'ink Floyd «The Wal', 'Columbia Records', 23, 1979)
(47, 'Billy Joel «Greatest Hits Volume I\xa0& Volume I', 'illy Joel «Greatest Hits Volume I\xa0& Volume I', 'Columbia Records', 23, 1985)
(44, 'AC/DC «Back in\xa0Blac', 'C/DC «Back in\xa0Blac', 'Epic Records', 22, 1980)
(43, 'Garth Brooks «Double Liv', 'arth Brooks «Double Liv', 'Capitol Nashville', 21, 1998)
(41, 'Shania Twain «Come on\xa0Ove', 'hania Twain «Come on\xa0Ove', 'Mercury Nashville', 20, 1987)
(42, 'Fleetwood Mac «Rumour', 'leetwood Mac «Rumour', 'Warner Bros.', 20, 1987)
(38, 'Whitney Houston «The Bodyguar', 'Whitney Houston «The Bodyguar', 'RCA', 18, 1992)
(39, 'Guns N’ Roses «Appetite for Destructio', 'Guns N’ Roses «Appetite for Destructio', 'Geffen Records', 18, 1987)


## Задачи для самостоятельного выполнения

**Самый самый**. По полученной БД, выяснить:
1. Самый свежий альбом
1. Сколько альбомов было выпущено в 1987 году.
1. Кто выпустил самый успешный альбом до 1987 года включительно? После 1987г?
1. Сколько всего копий альбомов 1987 года было продано?
1. Какой исполнитель выпустил больше всего альбомов?
1. В каком году было выпущено больше всего альбомов?
1. Альбомы какого года продавались лучше?

#Настройка

In [85]:
import pandas as pd
data = pd.read_excel('17_art.xls', index_col='Unnamed: 0')

In [86]:
import sqlite3
 
conn = sqlite3.connect("mydatabase.db")
cursor = conn.cursor()

In [88]:
data.to_sql('albums',conn, if_exists='replace')

50

#Задание 1

In [89]:
cursor.execute("""
SELECT title, max(year) FROM albums""" )

SQLres=cursor.fetchone()

In [92]:
print('Самый свежий альбом: ',SQLres[0])

Самый свежий альбом:  Adele «2


#Задание 2

In [108]:
cursor.execute("""
SELECT count(*) FROM albums 
WHERE year=1987""" )

SQLres=cursor.fetchone()

In [109]:
print('В 1987 году было выпущенно',SQLres[0], 'альбома')

В 1987 году было выпущенно 4 альбома


#Задание 3.1

In [95]:
cursor.execute("""
SELECT title, max(units_mln) FROM albums 
WHERE year<=1987""" )

SQLres=cursor.fetchone()

In [96]:
print('Выпустил самый успешный альбом до 1987 года включительно: ',SQLres[0])

Выпустил самый успешный альбом до 1987 года включительно:  ichael Jackson «Thrille


#Задание 3.2

In [97]:
cursor.execute("""
SELECT title, max(units_mln) FROM albums 
WHERE year>1987""" )

SQLres=cursor.fetchone()

In [99]:
print('Выпустил самый успешный альбом после 1987 года: ',SQLres[0])

Выпустил самый успешный альбом после 1987 года:  arth Brooks «Double Liv


#Задание 4

In [100]:
cursor.execute("""
SELECT sum(units_mln) FROM albums 
WHERE year=1987""" )

SQLres=cursor.fetchone()

In [101]:
print('Всего копий альбомов 1987 года было продано: ', SQLres[0])

Всего копий альбомов 1987 года было продано:  70


#Задание 5

Невозможно определить какой исполнитель выпустил больше всех альбомов, потому что в бд столбецы 'artist' и 'title' одинаковые. 

#Задание 6

In [102]:
cursor.execute("""SELECT year, max(cnt) FROM (SELECT year, count(title) as cnt FROM albums GROUP BY year)""" )

SQLres=cursor.fetchone()

In [104]:
print('Выпущено больше всего альбомов в', SQLres[0])

Выпущено больше всего альбомов в 1987


#Задание 7

In [105]:
cursor.execute("""SELECT year, max(sum) FROM (SELECT year, sum(units_mln) as sum FROM albums GROUP BY year)""" )

SQLres=cursor.fetchone()

In [106]:
print('Альбомы',SQLres[0],'года продавались лучше остальных')

Альбомы 1987 года продавались лучше остальных
